In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("MyApp") \
    .master("local[*]") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/06/15 11:11:48 WARN Utils: Your hostname, kirans-mac.local, resolves to a loopback address: 127.0.0.1; using 172.18.197.149 instead (on interface en0)
25/06/15 11:11:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/15 11:11:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# create dataFrame

In [3]:
data = (("James","Sales","NY",9000,34),
("Alicia","Sales","NY",8600,56),
("Robert","Sales","CA",8100,30),
("John","Sales","AZ",8600,31),
("Ross","Sales","AZ",8100,33),
("Kathy","Sales","AZ",1000,39),
("Lisa","Finance","CA",9000,24),
("Deja","Finance","CA",9900,40),
("Sugie","Finance","NY",8300,36),
("Ram","Finance","NY",7900,53),
("Satya","Finance","AZ",8200,53),
("Kyle","Marketing","CA",8000,25),
("Reid","Marketing","NY",9100,50)
)
schema=("empname","dept","state","salary","age")
df = spark.createDataFrame(data=data,schema=schema)
df.show()

+-------+---------+-----+------+---+
|empname|     dept|state|salary|age|
+-------+---------+-----+------+---+
|  James|    Sales|   NY|  9000| 34|
| Alicia|    Sales|   NY|  8600| 56|
| Robert|    Sales|   CA|  8100| 30|
|   John|    Sales|   AZ|  8600| 31|
|   Ross|    Sales|   AZ|  8100| 33|
|  Kathy|    Sales|   AZ|  1000| 39|
|   Lisa|  Finance|   CA|  9000| 24|
|   Deja|  Finance|   CA|  9900| 40|
|  Sugie|  Finance|   NY|  8300| 36|
|    Ram|  Finance|   NY|  7900| 53|
|  Satya|  Finance|   AZ|  8200| 53|
|   Kyle|Marketing|   CA|  8000| 25|
|   Reid|Marketing|   NY|  9100| 50|
+-------+---------+-----+------+---+



# Aggregate window functions

In [8]:
spec = Window.partitionBy("dept")

In [9]:
df.select(df.dept,df.salary) \
.withColumn("sum_sal",sum("salary").over(spec)) \
.withColumn("max_sal",max("salary").over(spec)) \
.withColumn("min_sal",min("salary").over(spec)) \
.withColumn("avg_sal",avg("salary").over(spec)) \
.withColumn("count_sal",count("salary").over(spec)) \
.show()

+---------+------+-------+-------+-------+-----------------+---------+
|     dept|salary|sum_sal|max_sal|min_sal|          avg_sal|count_sal|
+---------+------+-------+-------+-------+-----------------+---------+
|  Finance|  9000|  43300|   9900|   7900|           8660.0|        5|
|  Finance|  9900|  43300|   9900|   7900|           8660.0|        5|
|  Finance|  8300|  43300|   9900|   7900|           8660.0|        5|
|  Finance|  7900|  43300|   9900|   7900|           8660.0|        5|
|  Finance|  8200|  43300|   9900|   7900|           8660.0|        5|
|Marketing|  8000|  17100|   9100|   8000|           8550.0|        2|
|Marketing|  9100|  17100|   9100|   8000|           8550.0|        2|
|    Sales|  9000|  43400|   9000|   1000|7233.333333333333|        6|
|    Sales|  8600|  43400|   9000|   1000|7233.333333333333|        6|
|    Sales|  8100|  43400|   9000|   1000|7233.333333333333|        6|
|    Sales|  8600|  43400|   9000|   1000|7233.333333333333|        6|
|    S

In [10]:
spec = Window.partitionBy("dept").orderBy('salary')

df.select(df.dept,df.salary) \
.withColumn("first_sal",first("salary").over(spec)) \
.withColumn("last_sal",last("salary").over(spec)) \
.show()

+---------+------+---------+--------+
|     dept|salary|first_sal|last_sal|
+---------+------+---------+--------+
|  Finance|  7900|     7900|    7900|
|  Finance|  8200|     7900|    8200|
|  Finance|  8300|     7900|    8300|
|  Finance|  9000|     7900|    9000|
|  Finance|  9900|     7900|    9900|
|Marketing|  8000|     8000|    8000|
|Marketing|  9100|     8000|    9100|
|    Sales|  1000|     1000|    1000|
|    Sales|  8100|     1000|    8100|
|    Sales|  8100|     1000|    8100|
|    Sales|  8600|     1000|    8600|
|    Sales|  8600|     1000|    8600|
|    Sales|  9000|     1000|    9000|
+---------+------+---------+--------+



# Ranking window functions
- row_number() : Sequential Row Number.
- rank() : Ranks but gaps when ties.
- dense_rank() : Ranks with out any gaps.
- percent_rank: Relative rank (i.e. percentile) of rows within a window partition. First row is always 0 and last row is
always 1.
- ntile() :returns the ntile group id (from 1 to n inclusive) in an ordered window partition. For example, if n is 4, the first
quarter of the rows will get value 1, the second quarter will get 2, the third quarter will get 3, and the last quarter will
get 4.
- cume_dist():Returns the cumulative distribution of values within a window partition,
i.e. the fraction of rows that are below the current row.

In [11]:
from pyspark.sql.window import *
from pyspark.sql.functions import *
spec = Window.partitionBy("dept").orderBy("salary")

In [12]:
df.select('dept','salary') \
.withColumn("row_number",row_number().over(spec))\
.withColumn("rank",rank().over(spec))\
.withColumn("dense_rank",dense_rank().over(spec))\
.withColumn("percent_rank",percent_rank().over(spec))\
.withColumn("cume_dist",cume_dist().over(spec))\
.withColumn("ntile",ntile(3).over(spec))\
.show()

+---------+------+----------+----+----------+------------+-------------------+-----+
|     dept|salary|row_number|rank|dense_rank|percent_rank|          cume_dist|ntile|
+---------+------+----------+----+----------+------------+-------------------+-----+
|  Finance|  7900|         1|   1|         1|         0.0|                0.2|    1|
|  Finance|  8200|         2|   2|         2|        0.25|                0.4|    1|
|  Finance|  8300|         3|   3|         3|         0.5|                0.6|    2|
|  Finance|  9000|         4|   4|         4|        0.75|                0.8|    2|
|  Finance|  9900|         5|   5|         5|         1.0|                1.0|    3|
|Marketing|  8000|         1|   1|         1|         0.0|                0.5|    1|
|Marketing|  9100|         2|   2|         2|         1.0|                1.0|    2|
|    Sales|  1000|         1|   1|         1|         0.0|0.16666666666666666|    1|
|    Sales|  8100|         2|   2|         2|         0.2|       

# Analytical window functions
- lag() : Return offset row value before the current row value.
- lead() : Return offset row value after the current row value.

In [6]:
spec = Window.partitionBy("dept").orderBy("salary")

In [7]:
df.select('dept','salary') \
.withColumn("lag_prev_sal",lag("salary",1,0).over(spec)) \
.withColumn("lead_next_sal",lead("salary",1,0).over(spec)) \
.show()

+---------+------+------------+-------------+
|     dept|salary|lag_prev_sal|lead_next_sal|
+---------+------+------------+-------------+
|  Finance|  7900|           0|         8200|
|  Finance|  8200|        7900|         8300|
|  Finance|  8300|        8200|         9000|
|  Finance|  9000|        8300|         9900|
|  Finance|  9900|        9000|            0|
|Marketing|  8000|           0|         9100|
|Marketing|  9100|        8000|            0|
|    Sales|  1000|           0|         8100|
|    Sales|  8100|        1000|         8100|
|    Sales|  8100|        8100|         8600|
|    Sales|  8600|        8100|         8600|
|    Sales|  8600|        8600|         9000|
|    Sales|  9000|        8600|            0|
+---------+------+------------+-------------+



# Analytical windows functions
- rowbetween
- rangebetween

In [18]:
spec = Window.partitionBy("dept").orderBy("salary").rowsBetween(-1,1)

In [19]:
df.select('dept','salary')\
 .withColumn('sum_sal',sum('salary').over(spec))\
 .show()

+---------+------+-------+
|     dept|salary|sum_sal|
+---------+------+-------+
|  Finance|  7900|  16100|
|  Finance|  8200|  24400|
|  Finance|  8300|  25500|
|  Finance|  9000|  27200|
|  Finance|  9900|  18900|
|Marketing|  8000|  17100|
|Marketing|  9100|  17100|
|    Sales|  1000|   9100|
|    Sales|  8100|  17200|
|    Sales|  8100|  24800|
|    Sales|  8600|  25300|
|    Sales|  8600|  26200|
|    Sales|  9000|  17600|
+---------+------+-------+



In [38]:
spec = Window.partitionBy("dept").orderBy("salary").rowsBetween(Window.unboundedPreceding,Window.currentRow)

In [39]:
df.select('dept','salary')\
 .withColumn('sum_sal',sum('salary').over(spec))\
 .show()

+---------+------+-------+
|     dept|salary|sum_sal|
+---------+------+-------+
|  Finance|  7900|   7900|
|  Finance|  8200|  16100|
|  Finance|  8300|  24400|
|  Finance|  9000|  33400|
|  Finance|  9900|  43300|
|Marketing|  8000|   8000|
|Marketing|  9100|  17100|
|    Sales|  1000|   1000|
|    Sales|  8100|   9100|
|    Sales|  8100|  17200|
|    Sales|  8600|  25800|
|    Sales|  8600|  34400|
|    Sales|  9000|  43400|
+---------+------+-------+



In [40]:
spec = Window.partitionBy("dept").orderBy("salary").rangeBetween(Window.unboundedPreceding,Window.currentRow)

In [41]:
df.select('dept','salary')\
 .withColumn('sum_sal',sum('salary').over(spec))\
 .show()

+---------+------+-------+
|     dept|salary|sum_sal|
+---------+------+-------+
|  Finance|  7900|   7900|
|  Finance|  8200|  16100|
|  Finance|  8300|  24400|
|  Finance|  9000|  33400|
|  Finance|  9900|  43300|
|Marketing|  8000|   8000|
|Marketing|  9100|  17100|
|    Sales|  1000|   1000|
|    Sales|  8100|  17200|
|    Sales|  8100|  17200|
|    Sales|  8600|  34400|
|    Sales|  8600|  34400|
|    Sales|  9000|  43400|
+---------+------+-------+



- row between: https://www.youtube.com/watch?v=xLAmRI_Iky8&ab_channel=AshutoshKumar 
- row between vs range between: https://www.youtube.com/watch?v=4eEIs9gtKjw&ab_channel=DataTechWhiz (last 5 mins)

In [42]:
spark.stop()